In [1]:
!pip install transformers datasets evaluate accelerate

In [2]:
import torch
import pandas as pd
from torch.utils.data import DataLoader
from transformers import AutoTokenizer

In [3]:
class MyDataset(torch.utils.data.Dataset):
  def __init__(self, data):
    self.data = data

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    input_ids = self.data.iloc[idx]['tokens']['input_ids']
    att_mask = self.data.iloc[idx]['tokens']['attention_mask']
    label = 0 if 'non' in self.data.iloc[idx]['label'] else 1
    return {'input_ids': input_ids, 'attention_mask': att_mask, 'label': label}

In [4]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")
#tokenizer = AutoTokenizer.from_pretrained("roberta-base")

def preprocess_function(example):
  return tokenizer(str(example))

data = pd.read_csv('train.tsv', delimiter='\t', names=['id', 'text', 'label'])
data.set_index('id', inplace=True)
data['tokens'] = data['text'].apply(preprocess_function)

test_data = pd.read_csv('val.tsv', delimiter='\t', names=['id', 'text', 'label'])
test_data.set_index('id', inplace=True)
test_data['tokens'] = test_data['text'].apply(preprocess_function)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
dataset = MyDataset(data)
test_dataset = MyDataset(test_data)
#dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [6]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [7]:
import evaluate

accuracy = evaluate.load("accuracy")

In [8]:
import numpy as np

def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  predictions = np.argmax(predictions, axis=1)
  return accuracy.compute(predictions=predictions, references=labels)

In [9]:
id2label = {0: "non_joke", 1: "joke"}
label2id = {"non_joke": 0, "joke": 1}

In [10]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

#model = AutoModelForSequenceClassification.from_pretrained(
#    "roberta-base", num_labels=2, id2label=id2label, label2id=label2id
#)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
from huggingface_hub import notebook_login

notebook_login()

In [12]:
training_args = TrainingArguments(
    output_dir="my_model",
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.502526,0.780142
2,No log,0.467696,0.780142
3,No log,0.528340,0.787234


Checkpoint destination directory my_model/checkpoint-61 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory my_model/checkpoint-122 already exists and is non-empty. Saving will proceed but saved results may be invalid.


TrainOutput(global_step=183, training_loss=0.36481734312297215, metrics={'train_runtime': 1838.6634, 'train_samples_per_second': 1.57, 'train_steps_per_second': 0.1, 'total_flos': 71752311854040.0, 'train_loss': 0.36481734312297215, 'epoch': 3.0})

In [13]:
test = []

In [16]:
import csv

with open("public_test.tsv", mode='r', newline='', encoding='utf-8') as tsv_file:
  reader = csv.reader(tsv_file, delimiter='\t', skipinitialspace=True)
  next(reader)

  for row in reader:
    id = row[0]
    text = row[1]
    test.append({'ID': id, 'Text': text})

In [17]:
with open("private_test.tsv", mode='r', newline='', encoding='utf-8') as tsv_file:
  reader = csv.reader(tsv_file, delimiter='\t', skipinitialspace=True)
  next(reader)

  for row in reader:
    id = row[0]
    text = row[1]
    test.append({'ID': id, 'Text': text})

In [18]:
test_tokenizer = AutoTokenizer.from_pretrained("my_model")

In [19]:
model = AutoModelForSequenceClassification.from_pretrained("my_model")

In [20]:
res = []

for i in range(len(test)):
  sample = test[i]
  sample_input = test_tokenizer(sample['Text'], return_tensors="pt")

  with torch.no_grad():
    logits = model(**sample_input).logits
    predicted_class_id = logits.argmax().item()
    res.append({'ID': sample['ID'], 'Label': model.config.id2label[predicted_class_id]})

In [21]:
import csv

keys = res[0].keys()

with open('submission.csv', 'w', newline='') as output_file:
  writer = csv.DictWriter(output_file, keys)
  writer.writeheader()
  writer.writerows(res)